<a href="https://colab.research.google.com/github/MananShukla7/SkimLit/blob/main/skimlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project : SkimLit📝🔥

`Short for Skimming Literature`

The purpose of this notebook is to build a NLP model to make reading medical abstracts
a lot easier

The Dataset that we are using is PubMed 200k RCT and the paper we are replicating is :
https://arxiv.org/abs/1710.06071



##Confirming access to GPU

In [1]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


##Getting Data

Downloading the dataset from the paper's author: https://github.com/Franck-Dernoncourt/pubmed-rct

In [2]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct
!ls pubmed-rct

Cloning into 'pubmed-rct'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 33 (delta 5), reused 5 (delta 5), pack-reused 25
Unpacking objects: 100% (33/33), 177.08 MiB | 12.04 MiB/s, done.
Updating files: 100% (13/13), done.
PubMed_200k_RCT
PubMed_200k_RCT_numbers_replaced_with_at_sign
PubMed_20k_RCT
PubMed_20k_RCT_numbers_replaced_with_at_sign
README.md


In [3]:
#Check what filer are in the PubMed 20k dataset
!ls /content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/

dev.txt  test.txt  train.txt


In [4]:
#Start our experiments by exploring and experimenting on 20k dataset!
data_dir="/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign"

In [5]:
#Check all of the filename of its directory
import os
filenames=[data_dir+"/"+filename for filename in os.listdir(data_dir)]
filenames

['/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/dev.txt',
 '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/train.txt',
 '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/test.txt']

##Data Preprocessing

Now we've got some text data, now we have to explore it throughly.

To do that we need to visualise it first.

In [6]:
#Reading the files with python

def get_lines(filename):
  """
  Reads the filename (a text filename) and returns all of the lines of the text file
  as a list.

  Args:
  filname: a string containing the target filepath

  Returns:
  A list of string with one string per line from the input text line
  """
  f=open(filename,"r")
  return f.readlines()



In [7]:
# Let's read into the training lines
train_dir=filenames[1]
train_lines=get_lines(train_dir)
train_lines[:20]

['###24293578\n',
 'OBJECTIVE\tTo investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( OA ) .\n',
 'METHODS\tA total of @ patients with primary knee OA were randomized @:@ ; @ received @ mg/day of prednisolone and @ received placebo for @ weeks .\n',
 'METHODS\tOutcome measures included pain reduction and improvement in function scores and systemic inflammation markers .\n',
 'METHODS\tPain was assessed using the visual analog pain scale ( @-@ mm ) .\n',
 'METHODS\tSecondary outcome measures included the Western Ontario and McMaster Universities Osteoarthritis Index scores , patient global assessment ( PGA ) of the severity of knee OA , and @-min walk distance ( @MWD ) .\n',
 'METHODS\tSerum levels of interleukin @ ( IL-@ ) , IL-@ , tumor necrosis factor ( TNF ) - , and 

In [8]:
len(train_lines)

210040

###Data Structuring

Representing this data into dictionaries as it is easily manipulated.

A sample structure would look like:

```[{'line_number' : 0,
   'target' : 'BACKGROUND',
   'text' : 'Emotional eating is associated with overeating and the development of obesity .\n',
   'total_lines' : 11, 
   ...
   }]```

In [48]:
def preprocess_text_with_line_num(filename):
  types=["BACKGROUND", "OBJECTIVE", "METHODS", "RESULTS", "CONCLUSION"]
  list1=[]
  txtData=get_lines(filename)
  linecount=0
  for txt in txtData:
   
    if txt.startswith("###"):
        linecount=0
        start_index=txtData.index(txt)
        end_index=txtData.index("\n",start_index)
        total_lines=end_index-start_index
        # dict.update()
    
    elif txt.isspace():

      continue

    else:

      # for typ in types:
      #   if typ in txt:
      #     target=typ
      target="abc"


      start=txt.find("\t")
      end=txt.find(".\n")
      text=txt[start+1:end+1]
            
      
      data={}
      data["line_number"]=linecount
      data["target"]=target
      data["text"]=text
      data["total_line"]=total_lines
      # print(target.dtype)
      list1.append(data)
      linecount+=1
      # print(f"[line_number:{linecount},\ntarget:{target},\ntext:{text},\ntotal_line:{total_lines}]\n\n")
  return list1
  




  
    
    

In [49]:
filenames

['/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/dev.txt',
 '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/train.txt',
 '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/test.txt']

In [50]:
%%time
train_samples=preprocess_text_with_line_num(filenames[1])
val_samples=preprocess_text_with_line_num(filenames[0])
text_samples=preprocess_text_with_line_num(filenames[2])


CPU times: user 44.6 s, sys: 146 ms, total: 44.7 s
Wall time: 45.1 s


In [ ]:
str1="###35279E"
x="###" in str1
x

True

In [ ]:
l1=["BAKCGROUND","obj\n","\n"]
str2="BAKCGROUND \n ribdiuabda oddnaoid.\n"
for i in l1:
  print(i)
  if i in str2:
    x=i
s=str2.find("KC")
x,s

BAKCGROUND
obj





('\n', 2)

In [ ]:
str2[0:2]

'BA'

In [ ]:
l1.index("\n")

2

In [ ]:
td="""Recreate the above func of greater efficiency """